In [579]:
%matplotlib inline
import qiskit
from qiskit import IBMQ
from qiskit import Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QiskitError
from qiskit.quantum_info.operators import Operator
from qiskit.tools.visualization import circuit_drawer
from qiskit.tools.visualization import plot_histogram
from qiskit.tools.visualization import plot_state_city
from qiskit.providers.aer import noise
import random
from math import *
import math
import matplotlib
import numpy as np


In [580]:
## Initializing global variables____
# Quantum register is organized like the following:
# |t, x, g, c, a>, with (t+x) having n qubits (index+pattern), g having (n-1) qubits and c having 2 qubits
# Also, ancilla qubits (a) as support for mct gate

#找 AAA
#   AAA
#this is for the 2-D pattern matching
td_file = open("2D_intput.txt", "r")
td2_file = open("2D_intput.txt", "r")
text = td_file.read()

y_lines = 0 #總共有幾行
for line in td2_file:
    y_lines+=1

x_l = (len(text) - y_lines+1)/y_lines
x_lines = int(x_l)
seq_y = "AAA"
seq_y2 = "AAG"
seq_x = text[0:8]
seq_x2 = text[9:17]
seq_x3 = text[18:26]
print(seq_x)
print(seq_x2)
print(seq_x3)



# commonly for 1-D and 2-D pattern matching
Q_t = ceil(log2(len(seq_x))) # shifting
Q_x = len(seq_y) #contain the elements of the diagonal itself
Q_g = Q_t + Q_x - 1
Q_c = 2
Q_anc = 1
total_qubits = Q_t + Q_x + Q_g + Q_c + Q_anc

GGTAAAGT
TTAAAGGG
TTAGTGGG


In [581]:
## Initialization of IBM QX
#IBMQ.enable_account('INSERT TOKEN HERE')
#provider = IBMQ.get_provider()
# Pick an available backend
# If this isn’t available pick a backend whose name containes ’_qasm_simulator’ from the output above
#backend = provider.get_backend(’ibmq_qasm_simulator’)
backend = Aer.get_backend('qasm_simulator')

In [582]:
# Uncomment if you want to use local simulator
#backend= Aer.get_backend(’qasm_simulator’)
## Functions for recurrence dot matrix
def delta(x, y):
    return 0 if x == y else 1

In [583]:
def M(seq1, seq2, i, j, k):
    return sum(delta(x, y) for x, y in zip(seq1[i : i+k],seq2[j :j+k]))

In [584]:
def makeMatrix(seq1, seq2, k):
    n = len(seq1)
    m = len(seq2)
    return [[M(seq1, seq2, i, j, k) for j in range(m - k + 1)]
        for i in range(n - k + 1)]

In [585]:
def plotMatrix(M, t, seq1, seq2, nonblank = chr(0x25A0), blank = ' '):
    print(' |' + seq2)
    print('-' * (2 + len(seq2)))
    for label, row in zip(seq1, M):
        line = ''.join(nonblank if s < t else blank for s in row)
        print(label + '|' + line)
    return

In [586]:
def convertMatrix(M):
    for i in range(0, len(M)):
        for j in range(0, len(M[i])):
            if M[i][j] == 0:
                M[i][j] = 1
            elif M[i][j] == 1:
                M[i][j] = 0
    return M

In [587]:
def dotplot(seq1, seq2, k = 1, t = 1):
    if len(seq1) > len(seq2):
        raise Exception("Vertical sequence cannot be longer than horizontal sequence!")
    M = makeMatrix(seq1, seq2, k)
    plotMatrix(M, t, seq1, seq2)
    M = convertMatrix(M)
    return M

In [588]:
def getAllDiagonalsFromMatrix(M):
    D = np.array([])
    d_size = -1
    for i in range(0, len(M[0])):
        d = np.diag(M, k=i)
    
        if d_size == -1:
            d_size = len(d)
            D = d
        elif d_size > len(d):
            z = np.zeros((1, (d_size-len(d))), dtype=int)
            d = np.append(d, z)
            D = np.vstack((D, d))
        else:
            D = np.vstack((D, d))
    return D

In [589]:
def convertBinArrayToStr(array):
    string = ""
    for bin_digit in array:
        if bin_digit == 0:
            string = string + '0'
        elif bin_digit == 1:
            string = string + '1'
    return string

In [590]:
## Functions for Quantum Pattern Recognition
def generateInitialState(qc, qr, dot_matrix):
    D = getAllDiagonalsFromMatrix(dot_matrix)
    m = len(D)
    #print("Size of Learning Set: {}".format(len(D)))
    idx = 0
    for d in D:
        #print("{}->{}: {}".format(idx, format(idx,'0'+str(Q_t)+'b'), d))
        idx = idx + 1
    
    z_values = convertBinArrayToStr(np.zeros(Q_t+Q_x))
    
    ancilla_qubits = []
    for qi in range(0, Q_anc):
        ancilla_qubits.append(qr[Q_t + Q_x + Q_g + Q_c + qi])
    
    for p in range(m, 0, -1):
        bin_diagonal = convertBinArrayToStr(D[len(D)-p])
        index = format((len(D)-p), '0' + str(Q_t) + 'b')
        instance = index + bin_diagonal
        #print("Instance #{}, z={}".format(p, instance))
        
        #第三行到第五行
        for j in range(1, Q_t + Q_x + 1):
            if z_values[j-1] != instance[j-1]:
                #print("line 3-5: ")
                #print("F_0 #{} Applied to circuit with ctrl={} and target={}".format(j, Q_t+Q_x+Q_g+Q_c-1, j-1))
                qc.x(qr[Q_t+Q_x+Q_g+Q_c-1])
                qc.cx(qr[Q_t+Q_x+Q_g+Q_c-1], qr[j-1])
                qc.x(qr[Q_t+Q_x+Q_g+Q_c-1])
        #第六行 
        z_values = instance
        #print("line 6: ")
        #print("F_0 Applied to circuit with ctrl={} and target={}".format(Q_t+Q_x+Q_g+Q_c-1, Q_t+Q_x+Q_g+Q_c-2))
        qc.x(qr[Q_t+Q_x+Q_g+Q_c-1])
        qc.cx(qr[Q_t+Q_x+Q_g+Q_c-1], qr[Q_t+Q_x+Q_g+Q_c-2])
        qc.x(qr[Q_t+Q_x+Q_g+Q_c-1])
        
        #第七行
        #print("line 7: ")
        #print("S_{},{} Applied to circuit with ctrl={} and target={}".format(1, p, Q_t+Q_x+Q_g+Q_c-2, Q_t+Q_x+Q_g+Q_c-1))
        theta = 2*np.arcsin(1/sqrt(p))
        qc.cry(theta, qr[Q_t+Q_x+Q_g+Q_c-2], qr[Q_t+Q_x+Q_g+Q_c-1])
        
        if instance[0]=='0' and instance[1]=='0':
            #print("line 7-1: ")
            #print("A_00 #1 Applied to circuit with ctrl={},{} and target={}".format(0, 1, Q_t+Q_x))
            qc.x(qr[0])
            qc.x(qr[1])
            qc.mct([qr[0], qr[1]], qr[Q_t+Q_x], ancilla_qubits)
            qc.x(qr[1])
            qc.x(qr[0])
        elif instance[0]=='0' and instance[1]=='1':
            #print("line 7-2: ")
            #print("A_01 #1 Applied to circuit with ctrl={},{} and target={}".format(0, 1, Q_t+Q_x))
            qc.x(qr[0])
            qc.mct([qr[0], qr[1]], qr[Q_t+Q_x], ancilla_qubits)
            qc.x(qr[0])
        elif instance[0]=='1' and instance[1]=='0':
            #print("line 7-3: ")
            #print("A_10 #1 Applied to circuit with ctrl={},{} and target={}".format(0, 1, Q_t+Q_x))
            qc.x(qr[1])
            qc.mct([qr[0], qr[1]], qr[Q_t+Q_x], ancilla_qubits)
            qc.x(qr[1])
        elif instance[0]=='1' and instance[1]=='1':
            #print("line 7-4: ")
            #print("A_11 #1 Applied to circuit with ctrl={},{} and target={}".format(0, 1, Q_t+Q_x))
            qc.mct([qr[0], qr[1]], qr[Q_t+Q_x], ancilla_qubits)
        
        #第九行到第十行
        #print("line 9-10: ")
        for k in range(3, Q_t+Q_x+1):
            if instance[k-1]=='0':
                #print("A_01 #{} Applied to circuit with ctrl={},{} and target={}".format(k-1, k-1, Q_t+Q_x+k-3, Q_t+Q_x+k-2))
                qc.x(qr[k-1])
                qc.mct([qr[k-1], qr[Q_t+Q_x+k-3]], qr[Q_t+Q_x+k-2], ancilla_qubits)
                qc.x(qr[k-1])
            elif instance[k-1]=='1':
                #print("A_11 #{} Applied to circuit with ctrl={},{} and target={}".format(k-1, k-1, Q_t+Q_x+k-3, Q_t+Q_x+k-2))
                qc.mct([qr[k-1], qr[Q_t+Q_x+k-3]], qr[Q_t+Q_x+k-2], ancilla_qubits)
        #print("F_1 Applied to circuit with ctrl={} and target={}".format(Q_t+Q_x+Q_g-1, Q_t+Q_x+Q_g))
        #第十一行
       # print("line 11: ")
        qc.cx(qr[Q_t+Q_x+Q_g-1], qr[Q_t+Q_x+Q_g])
        
        #print("line 12-13: ")
        #第十二行到第十三行
        for k in range(Q_t+Q_x, 2, -1):
            if instance[k-1]=='0':
                #print("A_01 #{} Applied to circuit with ctrl={},{} and target={}".format(k-1, k-1, Q_t+Q_x+k-3, Q_t+Q_x+k-2))
                qc.x(qr[k-1])
                qc.mct([qr[k-1], qr[Q_t+Q_x+k-3]], qr[Q_t+Q_x+k-2], ancilla_qubits)
                qc.x(qr[k-1])
            elif instance[k-1]=='1':
                #print("A_11 #{} Applied to circuit with ctrl={},{} and target={}".format(k-1, k-1,Q_t+Q_x+k-3, Q_t+Q_x+k-2))
                qc.mct([qr[k-1], qr[Q_t+Q_x+k-3]], qr[Q_t+Q_x+k-2], ancilla_qubits)    
        #第十四行
        #print("line 14: ")
        if instance[0]=='0' and instance[1]=='0':
            #print("line 14-1: ")
            #print("A_00 #1 Applied to circuit with ctrl={},{} and target={}".format(0, 1, Q_t+Q_x))
            qc.x(qr[0])
            qc.x(qr[1])
            qc.mct([qr[0], qr[1]], qr[Q_t+Q_x], ancilla_qubits)
            qc.x(qr[1])
            qc.x(qr[0])
        elif instance[0]=='0' and instance[1]=='1':
            #print("line 14-2: ")
            #print("A_01 #1 Applied to circuit with ctrl={},{} and target={}".format(0, 1, Q_t+Q_x))
            qc.x(qr[0])
            qc.mct([qr[0], qr[1]], qr[Q_t+Q_x], ancilla_qubits)
            qc.x(qr[0])
        elif instance[0]=='1' and instance[1]=='0':
            #print("line 14-3: ")
            #print("A_10 #1 Applied to circuit with ctrl={},{} and target={}".format(0, 1, Q_t+Q_x))
            qc.x(qr[1])
            qc.mct([qr[0], qr[1]], qr[Q_t+Q_x], ancilla_qubits)
            qc.x(qr[1])
        elif instance[0]=='1' and instance[1]=='1':
            #print("line 14-5: ")
            #print("A_11 #1 Applied to circuit with ctrl={},{} and target={}".format(0, 1, Q_t+Q_x))
            qc.mct([qr[0], qr[1]], qr[Q_t+Q_x], ancilla_qubits)
    
    #print("F Applied to circuit at qubit={}".format(Q_t+Q_x+Q_g+Q_c-1))
    #第十五行
    #print("line 14: ")
    qc.x(qr[Q_t+Q_x+Q_g+Q_c-1])
    return    
            

In [591]:
def getIndices(mySet):
    indices = []
    for i in range(0, len(mySet)):
        tmp = ""
        for j in range(0, len(mySet[i])):
            tmp = tmp + str(int(mySet[i][j]))
        indices.append(int(tmp, 2))
    return indices

In [592]:
def oracle(query_set):
    I = np.identity(2**Q_x)
    b_sum = np.zeros((2**Q_x, 2**Q_x))
    
    indices = getIndices(query_set)
    for i in indices:
        vs = np.zeros((1, 2**Q_x))
        for j in range(0, 2**Q_x):
            if j == i:
                vs[0][j] = 1
        b_sum = b_sum + np.dot(np.conjugate(np.transpose(vs)), vs)
    U = I - (1-1j)*b_sum
    return U

In [593]:
def inversionAboutMean(dot_matrix):
    I = np.identity(2**(Q_t+Q_x))
    b_sum = np.zeros((2**(Q_t+Q_x), 1))
    
    D = getAllDiagonalsFromMatrix(dot_matrix)
    mySet = np.empty([len(D), Q_t+Q_x])
    
    for i in range(0, len(D)):
        bin_arr = np.concatenate((convertIntToBinArray(i, Q_t), D[i]))
        mySet[i] = bin_arr
        
    indices = getIndices(mySet)
    for i in indices:
        vs = np.zeros((2**(Q_t+Q_x), 1))
        for j in range(0, 2**(Q_t+Q_x)):
            if j == i:
                vs[j][0] = 1
        b_sum = b_sum + vs
    
    phi_zero = (1/sqrt(len(D))) * b_sum
    phi_mtrx = np.dot(phi_zero, np.conjugate(np.transpose(phi_zero)))
    U = (1 + 1j) * phi_mtrx - 1j * I
    return U

In [594]:
def convertIntToBinArray(j, dim):
    if not isinstance(j, int):
        raise Exception("Number of bits must be an integer!")
    elif (j == 0 or j == 1) and dim < 1:
        raise Exception("More bits needed to convert j in binary!")
    elif j > 1 and dim <= log2(j):
        raise Exception("More bits needed to convert j in binary!")
    
    bin_arr = np.array([], dtype=int)
    j_bin = format(int(j), '0' + str(dim) + 'b')
    
    for k in j_bin:
        if k == '1':
            bin_arr = np.append(bin_arr, 1)
        elif k == '0':
            bin_arr = np.append(bin_arr, 0)
    return bin_arr

In [595]:
def QPR_1D(dot_matrix):
    qr = qiskit.QuantumRegister(total_qubits)
    cr = qiskit.ClassicalRegister(Q_t)
    qc = qiskit.QuantumCircuit(qr, cr)
    
    print("Total number of qubits: {}".format(total_qubits))
    #print("Size of t register: {}".format(Q_t))
    #print("Size of x register: {}".format(Q_x))
    #print("Size of g register: {}".format(Q_g))
    #print("Size of c register: {}".format(Q_c))
    #print("Number of ancilla qubits: {}".format(Q_anc))
    
    # A query set is manually defined
    query_set = np.array([[1,1,1]])
    O_mtrx = oracle(query_set)
    U_phi_mtrx = inversionAboutMean(dot_matrix)
    O = Operator(O_mtrx)
    U_phi = Operator(U_phi_mtrx)
    
    O_qubits = []
    for qi in range(Q_x-1, -1, -1):
        O_qubits.append(Q_t + qi)
    
    U_phi_qubits = []
    for qi in range(Q_t+Q_x-1, -1, -1):
        U_phi_qubits.append(qi)
    
    generateInitialState(qc, qr, dot_matrix)
    #simulateStateVector(qc)
    
    T = round((math.pi/4)*sqrt(len(dot_matrix[0])/len(query_set)))
    
    it = 0
    for i in range(0, T):
        #print("Grover Iteration #{}".format(it+1))
        qc.unitary(O, O_qubits, label='O')
        #simulateStateVector(qc)
        
        qc.unitary(U_phi, U_phi_qubits, label='U_phi')
        #simulateStateVector(qc)
        it = it + 1


    #print("Grover’s algorithm had {} iterations.".format(int(it)))
    finalGroverMeasurement(qc, qr, cr)
    #print("Here is the graph of the quantum circuit")
    return qc

In [596]:
def simulateStateVector(qc):
    result = qiskit.execute(qc,backend=Aer.get_backend('statevector_simulator')).result()
    state = result.get_statevector(qc)
    print("Number of states in vector: {}".format(len(state)))
    it = 0
    for item in state:
        bin_str = format(it, '0'+str(total_qubits)+'b')
        bin_str_rev = bin_str[len(bin_str)::-1]
        if (item.real != 0 or item.imag != 0):
            print("{}->{}: {}".format(it, bin_str_rev[Q_t:Q_t+Q_x], item))
        it = it + 1
    return

In [597]:
# Final measurement
def finalGroverMeasurement(qc, qr, cr):
    for qi in range(0, Q_t):
        qc.measure(qr[qi], cr[qi])
    return

In [598]:
#try to do the 2-dimensional patternign mathching


In [602]:

## Main function
if __name__ == '__main__':
    #-----------------------第0行去隊第0大行-------------------------
    # Printing some data for testing
    M_0_0 = dotplot(seq_y, seq_x)
    qc_0_0 = QPR_1D(M_0_0)
    #print("Circuit depth: {}".format(qc_0_0.depth()))
    
    # Total number of gates
    print("Number of gates: {}".format(len(qc_0_0.data)))
    gate_num = 1
    for item in qc_0_0.data:
        qb_list = ""
        for qb in item[1]:
            qb_list = qb_list + str(qb.index) + ", "
        qb_list = qb_list[:len(qb_list)-2]
        #print("#{}: {}, {}".format(gate_num, item[0].name, qb_list))
        gate_num = gate_num + 1
        
    #-----------------------第0行去隊第1大行-------------------------
    M_0_1 = dotplot(seq_y, seq_x2)
    qc_0_1 = QPR_1D(M_0_1)
    #print("Circuit depth: {}".format(qc_0_1.depth()))
    
    # Total number of gates
    print("Number of gates: {}".format(len(qc_0_1.data)))
    gate_num = 1
    for item in qc_0_1.data:
        qb_list = ""
        for qb in item[1]:
            qb_list = qb_list + str(qb.index) + ", "
        qb_list = qb_list[:len(qb_list)-2]
        #print("#{}: {}, {}".format(gate_num, item[0].name, qb_list))
        gate_num = gate_num + 1
    #-----------------------第1行去隊第1大行-------------------------
    # Printing some data for testing
    M_1_1 = dotplot(seq_y2, seq_x2)
    qc_1_1 = QPR_1D(M_1_1)
    #print("Circuit depth: {}".format(qc_1_1.depth()))
    
    # Total number of gates
    print("Number of gates: {}".format(len(qc_1_1.data)))
    gate_num = 1
    for item in qc_1_1.data:
        qb_list = ""
        for qb in item[1]:
            qb_list = qb_list + str(qb.index) + ", "
        qb_list = qb_list[:len(qb_list)-2]
        #print("#{}: {}, {}".format(gate_num, item[0].name, qb_list))
        gate_num = gate_num + 1
        
    #-----------------------第1行去隊第2大行-------------------------
    M_1_2 = dotplot(seq_y2, seq_x3)
    qc_1_2 = QPR_1D(M_1_2)
    #print("Circuit depth: {}".format(qc_1_2.depth()))
    
    # Total number of gates
    print("Number of gates: {}".format(len(qc_1_2.data)))
    gate_num = 1
    for item in qc_1_2.data:
        qb_list = ""
        for qb in item[1]:
            qb_list = qb_list + str(qb.index) + ", "
        qb_list = qb_list[:len(qb_list)-2]
        #print("#{}: {}, {}".format(gate_num, item[0].name, qb_list))
        gate_num = gate_num + 1

#score=[[0]*40 for i in range(5)] // 宣告二維陣列名稱為score有5列40行。
compare_list = [[0]*2 for i in range(2)]
for i in range(2):
    for j in range(2):
        compare_list[i][j] ="222" #因為222不會出現
# Showing histogram
sim = qiskit.execute(qc_0_0, backend=backend, shots=1024)
result = sim.result()
final_0_0=result.get_counts(qc_0_0)
#print(final_0_0)
final_0_0_v = list(final_0_0.values())
final_0_0_k = list(final_0_0.keys())
plot_histogram(final_0_0)
#print(final_0_0_v)
#print(final_0_0_k)
#找最大的value存起來
#for i in final_0_0:

for i in range(8):
    if final_0_0_v[i] >= 500:
        compare_list[0][0] = final_0_0_k[i]
        #print("compare_list[0][0] = "+compare_list[0][0])
#------------------------------------------------
sim = qiskit.execute(qc_0_1, backend=backend, shots=1024)
result = sim.result()
final_0_1=result.get_counts(qc_0_1)
#print(final_0_1)
final_0_1_v = list(final_0_1.values())
final_0_1_k = list(final_0_1.keys())
plot_histogram(final_0_1)
#print(final_0_0_v)
#print(final_0_0_k)

for i in range(8):
    if final_0_1_v[i] >= 500:
        compare_list[0][1] = final_0_1_k[i]
        #print("compare_list[0][1] = "+compare_list[0][1])
#------------------------------------------------
sim = qiskit.execute(qc_1_1, backend=backend, shots=1024)
result = sim.result()
final_1_1=result.get_counts(qc_1_1)
#print(final_1_1)
final_1_1_v = list(final_1_1.values())
final_1_1_k = list(final_1_1.keys())
plot_histogram(final_1_1)
#print(final_1_1_v)
#print(final_1_1_k)

for i in range(8):
    if final_1_1_v[i] >= 500:
        compare_list[1][0] = final_1_1_k[i]
        #print("compare_list[1][0] = "+compare_list[1][0])
#------------------------------------------------
sim = qiskit.execute(qc_1_2, backend=backend, shots=1024)
result = sim.result()
final_1_2=result.get_counts(qc_1_2)
#print(final_1_2)
final_1_2_v = list(final_1_2.values())
final_1_2_k = list(final_1_2.keys())
plot_histogram(final_1_2)
#print(final_1_2_v)
#print(final_1_2_k)
for i in range(8):
    if final_1_2_v[i] >= 500:
        compare_list[1][1] = final_1_2_k[i]
        #print("compare_list[1][1] = "+compare_list[1][1])
#------------------------------------------------
#for i in range(2):
#    for j in range(2):
#        print("compare_list[i][j] = "+ compare_list[i][j])
#------------------------------------------------
#最後comparison
Ans = -1
for i in range (2):
    if compare_list[0][i] == compare_list[1][i]:
        Ans = 1
        print("the location for the first line is @:" + str(i) + "row, number { "+ compare_list[0][i] + " } bit")
#print(Ans)

 |GGTAAAGT
----------
A|   ■■■  
A|   ■■■  
A|   ■■■  
Total number of qubits: 14
Number of gates: 287
 |TTAAAGGG
----------
A|  ■■■   
A|  ■■■   
A|  ■■■   
Total number of qubits: 14
Number of gates: 287
 |TTAAAGGG
----------
A|  ■■■   
A|  ■■■   
G|     ■■■
Total number of qubits: 14
Number of gates: 287
 |TTAGTGGG
----------
A|  ■     
A|  ■     
G|   ■ ■■■
Total number of qubits: 14
Number of gates: 305
the location for the first line is @:0row, number { 110 } bit
